In [10]:
import os
import nltk
import pytesseract
import re
import slate3k as slate
from pdf2image import convert_from_path
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from PIL import Image
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/charan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/charan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
def extractText(file):
    pdfFileObj = open(file, "rb")
    print(pdfFileObj)
    pdfPages = slate.PDF(pdfFileObj)
    print(pdfPages)

    # Extract text from PDF file
    text = ""
    for page in pdfPages:
        text += page
    return text

In [27]:
text=extractText("texttospeech.pdf")

<_io.BufferedReader name='texttospeech.pdf'>
['See discussions, stats, and author profiles for this publication at : https://www .researchgate.net/publication/319406874\n\nText – T\n\no – Speech Synthesis ( T\n\nS)\n\nArticle · May 2014\n\nCITATIONS\n\n13\n\n3 author\n\ns:\n\nCosmas Ifeanyi Nwakanma\n\nKumoh National Institute of Technology\n\n144\n\n PUBLICATIONS\xa0\xa0\xa0510\n\n CITATIONS\xa0\xa0\xa0\n\nSEE PROFILE\n\nOkpala Izunna\n\nUniversity of Cincinnati\n\n18\n\n PUBLICATIONS\xa0\xa0\xa039\n\n CITATIONS\xa0\xa0\xa0\n\nSEE PROFILE\n\nREADS\n\n30,605\n\nIkenna Oluigbo\n\nClaude Bernard University Lyon 1\n\n14\n\n PUBLICATIONS\xa0\xa0\xa028\n\n CITATIONS\xa0\xa0\xa0\n\nSEE PROFILE\n\nSome of the authors of this publication are also working on these related projects:\n\nDESIGN AND IMPLEMENTATION OF TEXT-TO-SPEECH SYNTHESIS SYSTEM View project\n\nData-driven Intelligent Transportation Systems View project\n\nAll content following this page was uploaded by Cosmas Ifeanyi Nwakanma o

In [46]:
def summarize(text):
    # Process text by removing numbers and unrecognized punctuation
    processedText = re.sub("’", "'", text)
    processedText = re.sub("[^a-zA-Z' ]+", " ", processedText)
    stopWords = set(stopwords.words("english"))
    words = word_tokenize(processedText)
    print(words)

    #  Normalize words with Porter stemming and build word frequency table
    stemmer = SnowballStemmer("english", ignore_stopwords=True)
    freqTable = dict()
    for word in words:
        word = word.lower()
        if word in stopWords:
            continue
        elif stemmer.stem(word) in freqTable:
            freqTable[stemmer.stem(word)] += 1
        else:
            freqTable[stemmer.stem(word)] = 1
    
    print(freqTable)
    # # Normalize every sentence in the text
    sentences = sent_tokenize(text)
    stemmedSentences = []
    sentenceValue = dict()


    for sentence in sentences:
        stemmedSentence = []
        for word in sentence.lower().split():
            stemmedSentence.append(stemmer.stem(word))
        stemmedSentences.append(stemmedSentence)
    
    

    # # Calculate value of every normalized sentence based on word frequency table
    # [:12] helps to save space
    for num in range(len(stemmedSentences)):
        for wordValue in freqTable:
            if wordValue in stemmedSentences[num]:
                if sentences[num][:12] in sentenceValue:
                    sentenceValue[sentences[num][:12]] += freqTable.get(wordValue)
                else:
                    sentenceValue[sentences[num][:12]] = freqTable.get(wordValue)

    # # Determine average value of a sentence in the text
    sumValues = 0
    for sentence in sentenceValue:
        sumValues += sentenceValue.get(sentence)

    average = int(sumValues / len(sentenceValue))

    # Create summary of text using sentences that exceed the average value by some factor
    # This factor can be adjusted to reduce/expand the length of the summary
    summary = ""
    for sentence in sentences:
            if sentence[:12] in sentenceValue and sentenceValue[sentence[:12]] > (3.0 * average):
                summary += " " + " ".join(sentence.split())

    # # Process the text in summary and write it to a new file
    summary = re.sub("’", "'", summary)
    summary = re.sub("[^a-zA-Z0-9'\"():;,.!?— ]+", " ", summary)
    summaryText = open("summary.txt", "w")
    summaryText.write(summary)
    summaryText.close()
    print(summary)

In [47]:
summarize(text)

['See', 'discussions', 'stats', 'and', 'author', 'profiles', 'for', 'this', 'publication', 'at', 'https', 'www', 'researchgate', 'net', 'publication', 'Text', 'T', 'o', 'Speech', 'Synthesis', 'T', 'S', 'Article', 'May', 'CITATIONS', 'author', 's', 'Cosmas', 'Ifeanyi', 'Nwakanma', 'Kumoh', 'National', 'Institute', 'of', 'Technology', 'PUBLICATIONS', 'CITATIONS', 'SEE', 'PROFILE', 'Okpala', 'Izunna', 'University', 'of', 'Cincinnati', 'PUBLICATIONS', 'CITATIONS', 'SEE', 'PROFILE', 'READS', 'Ikenna', 'Oluigbo', 'Claude', 'Bernard', 'University', 'Lyon', 'PUBLICATIONS', 'CITATIONS', 'SEE', 'PROFILE', 'Some', 'of', 'the', 'authors', 'of', 'this', 'publication', 'are', 'also', 'working', 'on', 'these', 'related', 'projects', 'DESIGN', 'AND', 'IMPLEMENTATION', 'OF', 'TEXT', 'TO', 'SPEECH', 'SYNTHESIS', 'SYSTEM', 'View', 'project', 'Data', 'driven', 'Intelligent', 'Transportation', 'Systems', 'View', 'project', 'All', 'content', 'following', 'this', 'page', 'was', 'uploaded', 'by', 'Cosmas', 'I